In [6]:
import torch
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [7]:
# Rosenbrock vetorizado
def rosenbrock(x):
    """
    x: tensor de forma (N, 2)
    retorna: tensor de forma (N,)
    """
    return (1 - x[:, 0])**2 + 100 * (x[:, 1] - x[:, 0]**2)**2

In [8]:
def vectorized_gradient_descent(learning_rate, max_iter, initial_params, epsilon=1e-4):
    """
    Roda várias instâncias do gradiente descendente em paralelo.
    initial_params: tensor (N, 2)
    """
    x = initial_params.clone().detach().to(device).requires_grad_(True)

    for i in tqdm(range(max_iter)):
        if x.grad is not None:
            x.grad.zero_()

        # forward
        loss = rosenbrock(x).sum()  # soma total para backward

        # backward (GPU calcula todos os gradientes de uma vez)
        loss.backward()

        with torch.no_grad():
            grad_norm = torch.norm(x.grad, dim=1).mean()
            x -= learning_rate * x.grad

        if grad_norm < epsilon:
            print(f"Convergiu após {i + 1} iterações")
            break

    return x

In [9]:

def vectorized_heavy_ball(learning_rate, beta, max_iter, initial_params, epsilon=1e-4):
    x = initial_params.clone().detach().to(device).requires_grad_(True)
    prev_x = x.clone().detach()
    v = torch.zeros_like(x)

    for i in tqdm(range(max_iter)):
        if x.grad is not None:
            x.grad.zero_()

        loss = rosenbrock(x).sum()
        loss.backward()

        with torch.no_grad():
            v = beta * v + learning_rate * x.grad
            x -= v

        grad_norm = torch.norm(x.grad, dim=1).mean()
        if grad_norm < epsilon:
            print(f"Convergiu após {i + 1} iterações")
            break

    return x, i + 1


In [10]:
# 10.000 pontos iniciais diferentes
N = 10000
initial_params = torch.randn(N, 2, device=device)
learning_rates = np.arange(0.001, 0.1, 0.001)
beta = np.arange(0.5, 1.0, 0.01)
max_iter = 2000

best_lr = 0
best_beta = 0
least_iters = max_iter
num_iters = 0

for lr in learning_rates:
    for b in beta:
        final_points, num_iters = vectorized_heavy_ball(
            learning_rate=lr,
            beta=b,
            max_iter=max_iter,
            initial_params=initial_params
        )
        if num_iters < least_iters:
            least_iters = num_iters
            best_lr = lr
            best_beta = b

print(f"Melhor learning rate: {best_lr}, melhor beta: {best_beta}, número de iterações: {least_iters}")




  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

...

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 